In [ ]:
import allel
import pandas as pd
import numpy as np
import plotly.express as px

def load_vcf(vcf_path, metadata):
    """
    Load VCF and filter poor-quality samples
    """
        
    # load vcf and get genotypes and positions
    vcf = allel.read_vcf(vcf_path, fields='*')
    samples = vcf['samples']
    # keep only samples in qcpass metadata 
    sample_mask = np.isin(vcf['samples'], metadata.sampleID)
    
    # remove low quality samples 
    geno = allel.GenotypeArray(vcf['calldata/GT'])
    geno = geno.compress(sample_mask, axis=1)
    pos = vcf['variants/POS']
    contig = vcf['variants/CHROM']
    indel = vcf['variants/INDEL']
    
    # remove indels 
    geno = geno.compress(~indel, axis=0)
    pos = pos[~indel]
    contig = contig[~indel]
    
    return geno, pos, contig, samples[sample_mask]

In [ ]:
metadata_path = '../../../results/config/metadata.qcpass.tsv'
bed_targets_path = "../../../config/ag-vampir.bed"
vcf_path = "../../../results/vcfs/targets/ampseq-vigg01.annot.vcf"
wkdir = "../../.."
cohort_cols = 'taxon,location'

### Species ID

In [ ]:
metadata = pd.read_csv(metadata_path , sep="\t", index_col=0)
targets = pd.read_csv(bed_targets_path, sep="\t", header=None)
targets.columns = ['contig', 'start', 'end', 'amplicon', 'mutation']

geno, pos, contig, samples = load_vcf(vcf_path=vcf_path, metadata=metadata)
#ref = vcf['variants/REF']

In [ ]:
aim_targets = targets.query("mutation.str.contains('AIM')", engine='python')

aim_mask = np.isin(pos, aim_targets.end.to_list())
aim_gn = geno.compress(aim_mask, axis=0)
aim_pos = pos[aim_mask]
aim_contig = contig[aim_mask]

aim_df = pd.DataFrame(aim_gn.to_n_alt(fill=-1)).assign(pos=aim_pos, contig=aim_contig).set_index(['pos', 'contig'])
# reorder contigs 
aim_df = pd.concat([aim_df.query("contig == @contig") for contig in ['2R', '2L', '3R', '3L', 'X']])
aim_pos = aim_df.reset_index()['pos']
aim_contig = aim_df.reset_index()['contig']

x_label = [f"{c}:{p}" for c, p in zip(aim_contig, aim_pos)]

aim_df = aim_df.T
idxs = np.argsort(samples)
samples = samples[idxs]
aim_df = aim_df.iloc[idxs, :]

fig = px.imshow(
    aim_df.replace(-1, np.nan), 
    x=x_label,
    y=samples, 
    color_continuous_scale='Inferno',
    aspect='auto',
    title='gamb vs colu AIM genotypes',
    template='simple_white',
    width=1000, 
    height=800
)
fig.show()

### Species assignments by cohorts

In [ ]:
df = aim_df.T.query("contig != '2L'")
df.columns = samples
mean_aims = df.replace(-1, float('nan')).apply(np.nanmean, axis=0)
aims = mean_aims.loc[metadata.set_index('sampleID').index]
metadata = metadata.assign(mean_aim_genotype=aims.values)

fig = px.histogram(metadata, nbins=100, x='mean_aim_genotype', color='location', width=750, height=400, template='plotly_white', title='AIM genotype distribution')
fig.show()

In [ ]:

taxon = []
for i, row in metadata.iterrows():
    if row.mean_aim_genotype < 0.5:
        taxon.append('coluzzii')
    elif row.mean_aim_genotype >= 0.5 and row.mean_aim_genotype < 1.5:
        taxon.append('uncertain')
    elif row.mean_aim_genotype >= 1.5:
        taxon.append('gambiae')
    else: 
        taxon.append(np.nan)

from IPython.display import display, Markdown
new_metadata = metadata.assign(taxon=taxon)
new_metadata.to_csv(f"{wkdir}/results/config/metadata.qcpass.tsv", sep="\t")
new_metadata[['sampleID', 'taxon', 'mean_aim_genotype']].to_csv(f"{wkdir}/results/ag-vampir/aims/taxon_aims.tsv", sep="\t")
display(Markdown(f'<a href={wkdir}/results/ag-vampir/aims/taxon_aims.tsv>Sample aims and taxon assignment (.tsv)</a>'))
